In [1]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to c:\users\sec\appdata\local\temp\pip-install-0rd6b3ws\gym-walk_78e04f5ef93842fabfaa2b0e9792218a
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk 'C:\Users\SEC\AppData\Local\Temp\pip-install-0rd6b3ws\gym-walk_78e04f5ef93842fabfaa2b0e9792218a'


In [13]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [14]:

def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [15]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [24]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    np.random.seed(123)
    random.seed(123)
    env.reset(seed=123)
    env.action_space.seed(123)

    results = []
    for _ in range(n_episodes):
        state, _ = env.reset()
        for _ in range(max_steps):
            action = pi(state)  # <- CORRECT: pi is a function
            state, _, terminated, truncated, _ = env.step(action)
            if state == goal_state:
                results.append(1)
                break
            if terminated or truncated:
                results.append(0)
                break
    return np.mean(results)

In [25]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    np.random.seed(123)
    random.seed(123)
    env.reset(seed=123)
    env.action_space.seed(123)

    returns = []
    for _ in range(n_episodes):
        state, _ = env.reset()
        total_reward = 0
        for _ in range(max_steps):
            action = pi(state)
            state, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward
            if terminated or truncated:
                break
        returns.append(total_reward)
    return np.mean(returns)

In [26]:
desc=['FSFH','FFFH','FHGF','HFFH']
env = gym.make('FrozenLake-v1',desc=desc)
init_state = env.reset()
goal_state = 10
P = env.env.P

In [27]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, False),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, False),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},


In [28]:
def value_iteration(P, gamma=1.0, theta=1e-10):
    V = np.zeros(len(P), dtype=np.float64)
    while True:
      Q=np.zeros((len(P),len(P[0])),dtype=np.float64)
      for s in range(len(P)):
        for a in range(len(P[s])):
          for prob,next_state,reward,done in P[s][a]:
            Q[s][a]+=prob*(reward+gamma*V[next_state]*(not done))
      if np.max(np.abs(V-np.max(Q,axis=1)))<theta:
        break
      V=np.max(Q,axis=1)
    pi=lambda s:{s:a for s,a in enumerate(np.argmax(Q,axis=1))}[s]
    return V, pi

In [29]:
V_best_v, pi_best_v = value_iteration(P, gamma=0.99)

In [30]:
print("Name: SHALINI K   Register No.212222240095")
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_best_v, goal_state=goal_state) * 100,
    mean_return(env, pi_best_v)))

Name: SHALINI K   Register No.212222240095
Reaches goal 99.00%. Obtains an average undiscounted return of 0.9900.


In [31]:
# printing the success rate and the mean return
print("Name: SHALINI K   Register No.212222240095      ")
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_best_v, goal_state=goal_state)*100,
    mean_return(env, pi_best_v)))

Name: SHALINI K   Register No.212222240095      
Reaches goal 99.00%. Obtains an average undiscounted return of 0.9900.


In [32]:
print("Name: SHALINI K   Register No.212222240095      ")
print_state_value_function(V_best_v, P, prec=4)

Name: SHALINI K   Register No.212222240095      
State-value function:
| 00 0.7967 | 01 0.8189 | 02 0.8378 |           |
| 04 0.7987 | 05 0.8249 | 06  0.882 |           |
| 08 0.3934 |           |           | 11 0.4975 |
|           | 13 0.3235 | 14 0.6569 |           |
